# 🏭 Sistema PCP-TOC para Job Shop
## Planejamento e Controle da Produção com Teoria das Restrições

---

### 📋 O que este notebook faz:

✅ Identifica o **gargalo dinâmico** da produção  
✅ Calcula **buffers de tempo** (DBR - Drum-Buffer-Rope)  
✅ Prioriza pedidos por **penetração de buffer**  
✅ Gera **relatórios e gráficos** automáticos  

### 🚀 Como usar:

1. Execute a **Célula 1** (instala bibliotecas)
2. Execute a **Célula 2** (código principal)
3. Execute a **Célula 3** (exemplo básico)
4. Execute a **Célula 4** (gargalo dinâmico)

**Pronto!** Os resultados e gráficos aparecerão automaticamente! 📊

---

## 📦 Célula 1: Instalação de Dependências

Execute esta célula primeiro para instalar as bibliotecas necessárias.

In [ ]:
# Instalar dependências
!pip install -q pandas numpy matplotlib seaborn

print("✅ Bibliotecas instaladas com sucesso!")
print("📌 Agora execute a próxima célula (Célula 2)")

## 💻 Célula 2: Código Principal do Sistema PCP-TOC

Esta célula contém todo o código do sistema. Execute-a para carregar as classes e funções.

In [ ]:
"""
Sistema de PCP (Planejamento e Controle da Produção) para Job Shop
Baseado em TOC (Theory of Constraints) com Buffer Dinâmico de Tempo
e Identificação Dinâmica de Gargalos
"""

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dataclasses import dataclass, field
from typing import List, Dict, Optional, Tuple
from enum import Enum
import matplotlib.pyplot as plt
import seaborn as sns

class StatusPedido(Enum):
    PENDENTE = "Pendente"
    EM_PRODUCAO = "Em Produção"
    CONCLUIDO = "Concluído"
    ATRASADO = "Atrasado"

class TipoBuffer(Enum):
    SHIPPING = "Shipping Buffer"  # Buffer antes da entrega
    ASSEMBLY = "Assembly Buffer"  # Buffer antes de montagem
    CCR = "CCR Buffer"  # Buffer antes do recurso gargalo

@dataclass
class Operacao:
    """Representa uma operação em um recurso"""
    id_operacao: str
    recurso: str
    tempo_setup: float  # horas
    tempo_processamento: float  # horas
    ordem: int
    concluida: bool = False
    data_inicio: Optional[datetime] = None
    data_fim: Optional[datetime] = None

@dataclass
class Pedido:
    """Representa um pedido de produção"""
    id_pedido: str
    cliente: str
    operacoes: List[Operacao]
    data_entrada: datetime
    data_entrega: datetime
    prioridade: int = 5  # 1-10, onde 10 é mais prioritário
    status: StatusPedido = StatusPedido.PENDENTE
    buffer_tempo: float = 0.0  # em horas
    penetracao_buffer: float = 0.0  # % do buffer consumido
    
    def calcular_tempo_total(self) -> float:
        """Calcula tempo total de processamento"""
        return sum(op.tempo_setup + op.tempo_processamento for op in self.operacoes)
    
    def obter_proxima_operacao(self) -> Optional[Operacao]:
        """Retorna próxima operação não concluída"""
        for op in self.operacoes:
            if not op.concluida:
                return op
        return None

@dataclass
class Recurso:
    """Representa um recurso de produção"""
    nome: str
    capacidade_horas_dia: float = 8.0
    utilizacao_atual: float = 0.0
    fila: List[Tuple[str, str]] = field(default_factory=list)  # (id_pedido, id_operacao)
    carga_planejada: float = 0.0  # horas planejadas
    
    def calcular_utilizacao(self, horizonte_dias: int = 7) -> float:
        """Calcula % de utilização do recurso"""
        capacidade_total = self.capacidade_horas_dia * horizonte_dias
        return (self.carga_planejada / capacidade_total * 100) if capacidade_total > 0 else 0


class GestorGargalo:
    """Identifica e gerencia o gargalo dinâmico"""
    
    def __init__(self):
        self.historico_gargalos: List[Dict] = []
        self.recurso_gargalo_atual: Optional[str] = None
    
    def identificar_gargalo(self, recursos: Dict[str, Recurso], 
                           pedidos: List[Pedido],
                           horizonte_dias: int = 7) -> str:
        """
        Identifica o recurso gargalo baseado em:
        1. Maior utilização planejada
        2. Maior fila de trabalho
        3. Tempo total de processamento aguardando
        """
        metricas = {}
        
        for nome, recurso in recursos.items():
            # Calcular carga de trabalho futura
            carga_futura = 0
            num_ops_fila = 0
            
            for pedido in pedidos:
                if pedido.status != StatusPedido.CONCLUIDO:
                    for op in pedido.operacoes:
                        if op.recurso == nome and not op.concluida:
                            carga_futura += op.tempo_setup + op.tempo_processamento
                            num_ops_fila += 1
            
            recurso.carga_planejada = carga_futura
            utilizacao = recurso.calcular_utilizacao(horizonte_dias)
            
            metricas[nome] = {
                'utilizacao': utilizacao,
                'carga_horas': carga_futura,
                'num_operacoes': num_ops_fila,
                'score': utilizacao * 0.5 + (carga_futura / 10) * 0.3 + num_ops_fila * 0.2
            }
        
        # Identificar recurso com maior score
        if metricas:
            gargalo = max(metricas.items(), key=lambda x: x[1]['score'])
            self.recurso_gargalo_atual = gargalo[0]
            
            # Registrar no histórico
            self.historico_gargalos.append({
                'timestamp': datetime.now(),
                'recurso': gargalo[0],
                'metricas': metricas
            })
            
            return gargalo[0]
        
        return None


class GestorBuffer:
    """Gerencia os buffers de tempo baseado em TOC"""
    
    def __init__(self, tamanho_buffer_padrao: float = 24.0):
        """
        Args:
            tamanho_buffer_padrao: Tamanho padrão do buffer em horas
        """
        self.tamanho_buffer_padrao = tamanho_buffer_padrao
        self.buffer_shipping_multiplier = 1.0
        self.buffer_ccr_multiplier = 0.5
        self.buffer_assembly_multiplier = 0.75
    
    def calcular_buffer_pedido(self, pedido: Pedido, 
                               recurso_gargalo: str,
                               tipo_buffer: TipoBuffer = TipoBuffer.SHIPPING) -> float:
        """
        Calcula o buffer de tempo necessário para um pedido
        Buffer = Tempo de Processamento + Margem de Segurança
        """
        tempo_processamento = pedido.calcular_tempo_total()
        
        # Ajustar multiplicador baseado no tipo de buffer
        if tipo_buffer == TipoBuffer.SHIPPING:
            multiplicador = self.buffer_shipping_multiplier
        elif tipo_buffer == TipoBuffer.CCR:
            multiplicador = self.buffer_ccr_multiplier
        else:
            multiplicador = self.buffer_assembly_multiplier
        
        # Buffer proporcional ao tempo de processamento + buffer base
        buffer = (tempo_processamento * 0.5 + self.tamanho_buffer_padrao) * multiplicador
        
        return buffer
    
    def calcular_penetracao_buffer(self, pedido: Pedido, 
                                   data_atual: datetime) -> float:
        """
        Calcula a penetração do buffer (% consumido)
        0% = início do buffer, 100% = buffer totalmente consumido
        """
        if pedido.buffer_tempo <= 0:
            return 0.0
        
        # Tempo até a data de entrega
        tempo_restante = (pedido.data_entrega - data_atual).total_seconds() / 3600
        
        # Tempo já consumido do buffer
        tempo_consumido = pedido.buffer_tempo - tempo_restante
        
        # Penetração em %
        penetracao = (tempo_consumido / pedido.buffer_tempo) * 100
        
        return max(0, min(100, penetracao))
    
    def classificar_zona_buffer(self, penetracao: float) -> str:
        """
        Classifica a zona do buffer (Verde, Amarelo, Vermelho)
        Verde: 0-33%, Amarelo: 34-66%, Vermelho: 67-100%
        """
        if penetracao <= 33:
            return "VERDE"
        elif penetracao <= 66:
            return "AMARELO"
        else:
            return "VERMELHO"


class SistemaPCP_TOC:
    """Sistema principal de PCP baseado em TOC"""
    
    def __init__(self):
        self.pedidos: List[Pedido] = []
        self.recursos: Dict[str, Recurso] = {}
        self.gestor_gargalo = GestorGargalo()
        self.gestor_buffer = GestorBuffer()
        self.data_atual = datetime.now()
    
    def adicionar_recurso(self, nome: str, capacidade_horas_dia: float = 8.0):
        """Adiciona um recurso ao sistema"""
        self.recursos[nome] = Recurso(nome, capacidade_horas_dia)
    
    def adicionar_pedido(self, pedido: Pedido):
        """Adiciona um pedido ao sistema"""
        self.pedidos.append(pedido)
    
    def executar_planejamento(self):
        """
        Executa o planejamento completo baseado em TOC:
        1. Identificar o gargalo
        2. Subordinar tudo ao gargalo
        3. Calcular buffers
        4. Sequenciar operações
        """
        print("=" * 80)
        print("INICIANDO PLANEJAMENTO PCP - TOC")
        print("=" * 80)
        
        # Passo 1: Identificar gargalo
        print("\n[1] Identificando Recurso Gargalo...")
        recurso_gargalo = self.gestor_gargalo.identificar_gargalo(
            self.recursos, self.pedidos
        )
        print(f"✓ Gargalo identificado: {recurso_gargalo}")
        
        # Exibir métricas de todos os recursos
        print("\nMétricas dos Recursos:")
        for nome, recurso in self.recursos.items():
            util = recurso.calcular_utilizacao()
            marcador = " ← GARGALO" if nome == recurso_gargalo else ""
            print(f"  {nome}: {util:.1f}% utilização, "
                  f"{recurso.carga_planejada:.1f}h planejadas{marcador}")
        
        # Passo 2: Calcular buffers para todos os pedidos
        print("\n[2] Calculando Buffers de Tempo...")
        for pedido in self.pedidos:
            if pedido.status != StatusPedido.CONCLUIDO:
                pedido.buffer_tempo = self.gestor_buffer.calcular_buffer_pedido(
                    pedido, recurso_gargalo, TipoBuffer.SHIPPING
                )
                pedido.penetracao_buffer = self.gestor_buffer.calcular_penetracao_buffer(
                    pedido, self.data_atual
                )
                zona = self.gestor_buffer.classificar_zona_buffer(pedido.penetracao_buffer)
                print(f"  {pedido.id_pedido}: Buffer={pedido.buffer_tempo:.1f}h, "
                      f"Penetração={pedido.penetracao_buffer:.1f}% [{zona}]")
        
        # Passo 3: Priorizar pedidos baseado em penetração de buffer
        print("\n[3] Priorizando Pedidos (DBR - Drum-Buffer-Rope)...")
        pedidos_ativos = [p for p in self.pedidos if p.status != StatusPedido.CONCLUIDO]
        pedidos_priorizados = sorted(
            pedidos_ativos,
            key=lambda p: (-p.penetracao_buffer, p.prioridade),
            reverse=True
        )
        
        print("  Ordem de Prioridade:")
        for i, pedido in enumerate(pedidos_priorizados, 1):
            zona = self.gestor_buffer.classificar_zona_buffer(pedido.penetracao_buffer)
            print(f"  {i}. {pedido.id_pedido} - Penetração: {pedido.penetracao_buffer:.1f}% [{zona}]")
        
        # Passo 4: Sequenciar operações no gargalo primeiro
        print("\n[4] Sequenciando Operações no Gargalo...")
        self._sequenciar_gargalo(recurso_gargalo, pedidos_priorizados)
        
        # Passo 5: Subordinar demais recursos ao gargalo
        print("\n[5] Subordinando Demais Recursos ao Gargalo...")
        self._subordinar_recursos(recurso_gargalo, pedidos_priorizados)
        
        print("\n" + "=" * 80)
        print("PLANEJAMENTO CONCLUÍDO")
        print("=" * 80)
    
    def _sequenciar_gargalo(self, recurso_gargalo: str, pedidos: List[Pedido]):
        """Sequencia operações no recurso gargalo"""
        ops_gargalo = []
        
        for pedido in pedidos:
            for op in pedido.operacoes:
                if op.recurso == recurso_gargalo and not op.concluida:
                    ops_gargalo.append((pedido, op))
        
        print(f"  {len(ops_gargalo)} operações sequenciadas no gargalo {recurso_gargalo}")
        
        # Programar operações no gargalo
        tempo_acumulado = self.data_atual
        for i, (pedido, op) in enumerate(ops_gargalo, 1):
            op.data_inicio = tempo_acumulado
            op.data_fim = tempo_acumulado + timedelta(
                hours=op.tempo_setup + op.tempo_processamento
            )
            tempo_acumulado = op.data_fim
            print(f"    Op {i}: {pedido.id_pedido} - {op.id_operacao} "
                  f"({op.data_inicio.strftime('%d/%m %H:%M')} - "
                  f"{op.data_fim.strftime('%d/%m %H:%M')})")
    
    def _subordinar_recursos(self, recurso_gargalo: str, pedidos: List[Pedido]):
        """Subordina todos os recursos ao ritmo do gargalo"""
        for nome, recurso in self.recursos.items():
            if nome != recurso_gargalo:
                ops_recurso = []
                for pedido in pedidos:
                    for op in pedido.operacoes:
                        if op.recurso == nome and not op.concluida:
                            ops_recurso.append((pedido, op))
                
                if ops_recurso:
                    print(f"  {nome}: {len(ops_recurso)} operações subordinadas")
    
    def gerar_relatorio_buffer(self) -> pd.DataFrame:
        """Gera relatório de status dos buffers"""
        dados = []
        
        for pedido in self.pedidos:
            if pedido.status != StatusPedido.CONCLUIDO:
                zona = self.gestor_buffer.classificar_zona_buffer(pedido.penetracao_buffer)
                dados.append({
                    'Pedido': pedido.id_pedido,
                    'Cliente': pedido.cliente,
                    'Buffer (h)': round(pedido.buffer_tempo, 1),
                    'Penetração (%)': round(pedido.penetracao_buffer, 1),
                    'Zona': zona,
                    'Prioridade': pedido.prioridade,
                    'Data Entrega': pedido.data_entrega.strftime('%d/%m/%Y'),
                    'Status': pedido.status.value
                })
        
        return pd.DataFrame(dados)
    
    def gerar_relatorio_recursos(self) -> pd.DataFrame:
        """Gera relatório de utilização dos recursos"""
        dados = []
        
        for nome, recurso in self.recursos.items():
            is_gargalo = nome == self.gestor_gargalo.recurso_gargalo_atual
            dados.append({
                'Recurso': nome,
                'Capacidade (h/dia)': recurso.capacidade_horas_dia,
                'Carga Planejada (h)': round(recurso.carga_planejada, 1),
                'Utilização (%)': round(recurso.calcular_utilizacao(), 1),
                'É Gargalo': 'SIM' if is_gargalo else 'NÃO'
            })
        
        return pd.DataFrame(dados)
    
    def visualizar_buffers(self):
        """Visualiza status dos buffers"""
        df = self.gerar_relatorio_buffer()
        
        if df.empty:
            print("Nenhum pedido ativo para visualizar")
            return
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
        
        # Gráfico 1: Penetração de Buffer por Pedido
        cores = {'VERDE': 'green', 'AMARELO': 'yellow', 'VERMELHO': 'red'}
        cores_barras = [cores[z] for z in df['Zona']]
        
        ax1.barh(df['Pedido'], df['Penetração (%)'], color=cores_barras, alpha=0.7)
        ax1.axvline(x=33, color='green', linestyle='--', alpha=0.5, label='Zona Verde')
        ax1.axvline(x=66, color='orange', linestyle='--', alpha=0.5, label='Zona Amarela')
        ax1.set_xlabel('Penetração do Buffer (%)')
        ax1.set_ylabel('Pedido')
        ax1.set_title('Penetração de Buffer por Pedido (TOC)')
        ax1.legend()
        ax1.grid(axis='x', alpha=0.3)
        
        # Gráfico 2: Distribuição por Zona
        zona_counts = df['Zona'].value_counts()
        ax2.pie(zona_counts.values, labels=zona_counts.index, autopct='%1.1f%%',
                colors=[cores[z] for z in zona_counts.index])
        ax2.set_title('Distribuição de Pedidos por Zona de Buffer')
        
        plt.tight_layout()
        plt.show()
        print("✓ Gráfico de buffers gerado")
    
    def visualizar_carga_recursos(self):
        """Visualiza carga dos recursos"""
        df = self.gerar_relatorio_recursos()
        
        fig, ax = plt.subplots(figsize=(10, 6))
        
        cores = ['red' if g == 'SIM' else 'steelblue' for g in df['É Gargalo']]
        
        x = range(len(df))
        width = 0.35
        
        ax.bar([i - width/2 for i in x], df['Utilização (%)'], width, 
               label='Utilização (%)', color=cores, alpha=0.7)
        ax.bar([i + width/2 for i in x], 
               (df['Carga Planejada (h)'] / df['Capacidade (h/dia)'] / 7 * 100), 
               width, label='Carga Planejada (%)', color='orange', alpha=0.5)
        
        ax.set_xlabel('Recursos')
        ax.set_ylabel('Percentual (%)')
        ax.set_title('Utilização e Carga dos Recursos (Gargalo em Vermelho)')
        ax.set_xticks(x)
        ax.set_xticklabels(df['Recurso'], rotation=45)
        ax.legend()
        ax.axhline(y=100, color='red', linestyle='--', alpha=0.3, label='100% Capacidade')
        ax.grid(axis='y', alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        print("✓ Gráfico de recursos gerado")

print("✅ Sistema PCP-TOC carregado com sucesso!")
print("📌 Agora execute a próxima célula (Célula 3 ou 4)")

## 🎯 Célula 3: Exemplo Básico (5 Pedidos)

Execute esta célula para ver o sistema funcionando com um exemplo completo.

In [ ]:
# Criar sistema
sistema = SistemaPCP_TOC()

# Adicionar recursos
sistema.adicionar_recurso('Corte', 8.0)
sistema.adicionar_recurso('Dobra', 8.0)
sistema.adicionar_recurso('Solda', 6.0)  # Capacidade reduzida - possível gargalo
sistema.adicionar_recurso('Pintura', 8.0)
sistema.adicionar_recurso('Montagem', 8.0)

# Criar pedidos
data_base = datetime.now()

# Pedido 1
pedido1 = Pedido(
    id_pedido="PED-001",
    cliente="Cliente A",
    operacoes=[
        Operacao("OP-001-1", "Corte", 0.5, 2.0, 1),
        Operacao("OP-001-2", "Dobra", 0.3, 1.5, 2),
        Operacao("OP-001-3", "Solda", 0.5, 3.0, 3),
        Operacao("OP-001-4", "Pintura", 0.2, 2.0, 4),
        Operacao("OP-001-5", "Montagem", 0.5, 2.5, 5),
    ],
    data_entrada=data_base,
    data_entrega=data_base + timedelta(days=5),
    prioridade=8
)

# Pedido 2
pedido2 = Pedido(
    id_pedido="PED-002",
    cliente="Cliente B",
    operacoes=[
        Operacao("OP-002-1", "Corte", 0.3, 1.0, 1),
        Operacao("OP-002-2", "Solda", 0.4, 2.5, 2),
        Operacao("OP-002-3", "Pintura", 0.2, 1.5, 3),
    ],
    data_entrada=data_base,
    data_entrega=data_base + timedelta(days=3),
    prioridade=10
)

# Pedido 3
pedido3 = Pedido(
    id_pedido="PED-003",
    cliente="Cliente C",
    operacoes=[
        Operacao("OP-003-1", "Corte", 0.4, 1.5, 1),
        Operacao("OP-003-2", "Dobra", 0.3, 1.0, 2),
        Operacao("OP-003-3", "Solda", 0.5, 2.0, 3),
        Operacao("OP-003-4", "Montagem", 0.3, 1.5, 4),
    ],
    data_entrada=data_base,
    data_entrega=data_base + timedelta(days=7),
    prioridade=5
)

# Pedido 4
pedido4 = Pedido(
    id_pedido="PED-004",
    cliente="Cliente D",
    operacoes=[
        Operacao("OP-004-1", "Corte", 0.3, 1.0, 1),
        Operacao("OP-004-2", "Solda", 0.6, 4.0, 2),
        Operacao("OP-004-3", "Pintura", 0.2, 1.0, 3),
    ],
    data_entrada=data_base,
    data_entrega=data_base + timedelta(days=4),
    prioridade=7
)

# Pedido 5
pedido5 = Pedido(
    id_pedido="PED-005",
    cliente="Cliente E",
    operacoes=[
        Operacao("OP-005-1", "Corte", 0.2, 0.5, 1),
        Operacao("OP-005-2", "Dobra", 0.2, 0.8, 2),
        Operacao("OP-005-3", "Solda", 0.3, 1.5, 3),
    ],
    data_entrada=data_base,
    data_entrega=data_base + timedelta(days=6),
    prioridade=4
)

# Adicionar pedidos
sistema.adicionar_pedido(pedido1)
sistema.adicionar_pedido(pedido2)
sistema.adicionar_pedido(pedido3)
sistema.adicionar_pedido(pedido4)
sistema.adicionar_pedido(pedido5)

# Executar planejamento
sistema.executar_planejamento()

# Gerar relatórios
print("\n" + "=" * 80)
print("RELATÓRIOS")
print("=" * 80)

print("\n📊 RELATÓRIO DE BUFFERS:")
display(sistema.gerar_relatorio_buffer())

print("\n📊 RELATÓRIO DE RECURSOS:")
display(sistema.gerar_relatorio_recursos())

# Gerar visualizações
print("\n📈 Gerando visualizações...")
sistema.visualizar_buffers()
sistema.visualizar_carga_recursos()

print("\n✅ Exemplo básico concluído!")

## 🔄 Célula 4: Demonstração de Gargalo Dinâmico (3 Cenários)

Execute esta célula para ver como o gargalo muda conforme o mix de produtos.

In [ ]:
print("\n" + "=" * 90)
print("DEMONSTRAÇÃO: GARGALO DINÂMICO EM JOB SHOP")
print("=" * 90)

# ============================================
# CENÁRIO 1: Mix com muita SOLDA
# ============================================
print("\n" + "▓" * 90)
print("CENÁRIO 1: Mix de Produtos com Muita Solda")
print("▓" * 90)

sistema1 = SistemaPCP_TOC()
sistema1.adicionar_recurso('Corte', 8.0)
sistema1.adicionar_recurso('Dobra', 8.0)
sistema1.adicionar_recurso('Solda', 6.0)
sistema1.adicionar_recurso('Pintura', 8.0)

data_base = datetime.now()

# Pedidos com muita solda
sistema1.adicionar_pedido(Pedido("PED-001", "Cliente A", [
    Operacao("OP1", "Corte", 0.3, 1.0, 1),
    Operacao("OP2", "Solda", 0.5, 4.0, 2),
    Operacao("OP3", "Pintura", 0.2, 1.0, 3),
], data_base, data_base + timedelta(days=4), 8))

sistema1.adicionar_pedido(Pedido("PED-002", "Cliente B", [
    Operacao("OP1", "Corte", 0.3, 0.8, 1),
    Operacao("OP2", "Solda", 0.4, 3.5, 2),
], data_base, data_base + timedelta(days=3), 10))

sistema1.adicionar_pedido(Pedido("PED-003", "Cliente C", [
    Operacao("OP1", "Dobra", 0.3, 1.0, 1),
    Operacao("OP2", "Solda", 0.5, 3.0, 2),
    Operacao("OP3", "Pintura", 0.2, 1.5, 3),
], data_base, data_base + timedelta(days=5), 7))

sistema1.executar_planejamento()
print("\n📊 Resultado Cenário 1:")
display(sistema1.gerar_relatorio_recursos())

# ============================================
# CENÁRIO 2: Mix com muita DOBRA
# ============================================
print("\n\n" + "▓" * 90)
print("CENÁRIO 2: Mix de Produtos com Muita Dobra")
print("▓" * 90)

sistema2 = SistemaPCP_TOC()
sistema2.adicionar_recurso('Corte', 8.0)
sistema2.adicionar_recurso('Dobra', 8.0)
sistema2.adicionar_recurso('Solda', 6.0)
sistema2.adicionar_recurso('Pintura', 8.0)

# Pedidos com muita dobra
sistema2.adicionar_pedido(Pedido("PED-004", "Cliente D", [
    Operacao("OP1", "Corte", 0.3, 1.0, 1),
    Operacao("OP2", "Dobra", 0.5, 4.5, 2),
    Operacao("OP3", "Pintura", 0.2, 1.0, 3),
], data_base, data_base + timedelta(days=4), 8))

sistema2.adicionar_pedido(Pedido("PED-005", "Cliente E", [
    Operacao("OP1", "Corte", 0.3, 0.8, 1),
    Operacao("OP2", "Dobra", 0.4, 3.8, 2),
    Operacao("OP3", "Solda", 0.3, 1.0, 3),
], data_base, data_base + timedelta(days=3), 10))

sistema2.adicionar_pedido(Pedido("PED-006", "Cliente F", [
    Operacao("OP1", "Dobra", 0.5, 4.0, 1),
    Operacao("OP2", "Pintura", 0.2, 1.5, 2),
], data_base, data_base + timedelta(days=5), 7))

sistema2.adicionar_pedido(Pedido("PED-007", "Cliente G", [
    Operacao("OP1", "Corte", 0.3, 1.2, 1),
    Operacao("OP2", "Dobra", 0.4, 3.5, 2),
    Operacao("OP3", "Solda", 0.2, 0.8, 3),
], data_base, data_base + timedelta(days=6), 6))

sistema2.executar_planejamento()
print("\n📊 Resultado Cenário 2:")
display(sistema2.gerar_relatorio_recursos())

# ============================================
# CENÁRIO 3: Equipamento QUEBRADO (Pintura)
# ============================================
print("\n\n" + "▓" * 90)
print("CENÁRIO 3: Equipamento de Pintura Quebrado")
print("▓" * 90)

sistema3 = SistemaPCP_TOC()
sistema3.adicionar_recurso('Corte', 8.0)
sistema3.adicionar_recurso('Dobra', 8.0)
sistema3.adicionar_recurso('Solda', 6.0)
sistema3.adicionar_recurso('Pintura', 3.0)  # QUEBRADO - apenas 3h/dia

# Pedidos variados
sistema3.adicionar_pedido(Pedido("PED-008", "Cliente H", [
    Operacao("OP1", "Corte", 0.3, 1.0, 1),
    Operacao("OP2", "Solda", 0.3, 2.0, 2),
    Operacao("OP3", "Pintura", 0.3, 2.5, 3),
], data_base, data_base + timedelta(days=4), 9))

sistema3.adicionar_pedido(Pedido("PED-009", "Cliente I", [
    Operacao("OP1", "Dobra", 0.3, 1.0, 1),
    Operacao("OP2", "Pintura", 0.3, 3.0, 2),
], data_base, data_base + timedelta(days=3), 10))

sistema3.adicionar_pedido(Pedido("PED-010", "Cliente J", [
    Operacao("OP1", "Corte", 0.2, 0.8, 1),
    Operacao("OP2", "Dobra", 0.3, 1.2, 2),
    Operacao("OP3", "Pintura", 0.3, 2.8, 3),
], data_base, data_base + timedelta(days=5), 8))

sistema3.executar_planejamento()
print("\n📊 Resultado Cenário 3:")
display(sistema3.gerar_relatorio_recursos())

# ============================================
# ANÁLISE COMPARATIVA
# ============================================
print("\n\n" + "=" * 90)
print("ANÁLISE COMPARATIVA: EVOLUÇÃO DO GARGALO")
print("=" * 90)

print(f"""
╔═══════════════════════════════════════════════════════════════════════════════╗
║  CENÁRIO                           │ GARGALO        │ UTILIZAÇÃO  │ MOTIVO    ║
╠═══════════════════════════════════════════════════════════════════════════════╣
║  1. Mix com muita solda            │ SOLDA          │ {sistema1.recursos['Solda'].calcular_utilizacao():>5.1f}%      │ Demanda   ║
║  2. Mix com muita dobra            │ DOBRA          │ {sistema2.recursos['Dobra'].calcular_utilizacao():>5.1f}%      │ Demanda   ║
║  3. Pintura com equipamento quebrado│ PINTURA        │ {sistema3.recursos['Pintura'].calcular_utilizacao():>5.1f}%      │ Capacidade║
╚═══════════════════════════════════════════════════════════════════════════════╝
""")

print("\n🔍 CONCLUSÕES:")
print("""
1. O GARGALO É DINÂMICO: Muda conforme:
   • Mix de produtos varia (demanda por diferentes operações)
   • Capacidade dos recursos muda (quebras, manutenção, turnos)
   • Novos pedidos entram no sistema

2. IMPORTÂNCIA DA IDENTIFICAÇÃO CONTÍNUA:
   • Sistema deve recalcular gargalo regularmente
   • Decisões de investimento baseadas no gargalo atual
   • Buffers dimensionados para proteger o gargalo identificado

3. SUBORDINAÇÃO É FUNDAMENTAL:
   • Quando gargalo muda, toda a fábrica deve se adaptar
   • Recursos não-gargalo devem ter folga (não 100% utilizados)
   • Elevar capacidade do gargalo pode mudar qual é o gargalo

4. GESTÃO DE BUFFER PROTEGE O SISTEMA:
   • Buffer dimensionado baseado no gargalo atual
   • Penetração de buffer indica prioridades reais
   • Sistema responde rápido a mudanças
""")

print("\n✅ Demonstração de gargalo dinâmico concluída!")
print("=" * 90)

## 🔧 Célula 5: Customização - Crie Seu Próprio Cenário

Use esta célula para criar seus próprios pedidos e recursos!

In [ ]:
# Crie seu próprio sistema aqui!

meu_sistema = SistemaPCP_TOC()

# Adicione seus recursos
meu_sistema.adicionar_recurso('Recurso1', 8.0)
meu_sistema.adicionar_recurso('Recurso2', 8.0)
# ... adicione mais recursos

# Crie seus pedidos
data_hoje = datetime.now()

meu_pedido = Pedido(
    id_pedido="MEU-PED-001",
    cliente="Meu Cliente",
    operacoes=[
        Operacao("OP1", "Recurso1", tempo_setup=0.5, tempo_processamento=2.0, ordem=1),
        Operacao("OP2", "Recurso2", tempo_setup=0.3, tempo_processamento=1.5, ordem=2),
        # ... adicione mais operações
    ],
    data_entrada=data_hoje,
    data_entrega=data_hoje + timedelta(days=5),
    prioridade=8
)

meu_sistema.adicionar_pedido(meu_pedido)
# ... adicione mais pedidos

# Execute o planejamento
meu_sistema.executar_planejamento()

# Veja os resultados
print("\n📊 Relatório de Buffers:")
display(meu_sistema.gerar_relatorio_buffer())

print("\n📊 Relatório de Recursos:")
display(meu_sistema.gerar_relatorio_recursos())

# Gere os gráficos
meu_sistema.visualizar_buffers()
meu_sistema.visualizar_carga_recursos()

---

## 📚 Documentação Adicional

### Conceitos TOC Implementados:

1. **Identificação do Gargalo**: Sistema calcula automaticamente qual recurso limita a produção
2. **DBR (Drum-Buffer-Rope)**: Gargalo determina ritmo, buffer protege, corda controla liberação
3. **Gestão de Buffer**: Monitoramento em 3 zonas (Verde/Amarelo/Vermelho)
4. **Subordinação**: Recursos não-gargalo trabalham no ritmo do gargalo

### Referências:

- **Livro**: "A Meta" - Eliyahu Goldratt
- **Site**: tocico.org
- **Documentação**: Veja os arquivos README.md e DOCUMENTACAO_PCP_TOC.md

---

**Desenvolvido com base nos princípios de TOC de Eliyahu Goldratt** 🎯

✅ Sistema pronto para uso no Google Colab!